In [33]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import re
import string
import unicodedata
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vassi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In order to download the 'train.csv' and 'test.csv' files, follow the link: https://drive.google.com/drive/folders/1GtFMaqWYieUR9A6D3gO_u8QjsBVkIi8H?usp=sharing

In [26]:
location = 'C:/Users/vassi/Desktop/'
train = pd.read_csv(location+'train.csv')
test = pd.read_csv(location+'test.csv')
train = train[~((train['toxic']==0) & (train['severe_toxic']==0) & (train['obscene']==0) & (train['threat']==0) & (train['insult']==0) & (train['identity_hate']==0))]
train = train.reset_index().drop(['index', 'id'], axis=1)
test = test.reset_index()

In [23]:
def remove_stopwords(text):
    sw_nltk = stopwords.words('english')
    sw_nltk.append('st')
    sw_nltk.append('nd')
    sw_nltk.append('rd')
    sw_nltk.append('th')
    sw_nltk.append('rt')
    words = [word for word in text.split() if word.lower() not in sw_nltk]
    new_text = " ".join(words)
    return new_text

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation'''
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = ''.join(i for i in text if not i.isdigit())
    return text

In [28]:
# delete nan messages and clean the messages

train=train[~train['comment_text'].isna()]
train['comment_text']=train['comment_text'].apply(lambda x: clean_text(x))

#remove empty comments and stopwords
train = train[~train['comment_text'].isin(['', ' ', np.nan, None, []])]
train['comment_text']=train['comment_text'].apply(lambda x: remove_stopwords(x))

#remove empty comments after the removal of stopwords
train = train[~train['comment_text'].isin(['', ' ', np.nan, None, []])]
# df.to_csv(location+'Chat.csv')

In [34]:
X_train, X_test = train_test_split(train, random_state=420, test_size=0.2, shuffle=True)
y_train = X_train.drop(['comment_text'],axis=1)
X_train = X_train.comment_text
y_test = X_test.drop(['comment_text'],axis=1)
X_test = X_test.comment_text

In [39]:
# Define a pipeline combining a text feature extractor with multi lable classifier

df_toxic = train.drop(['comment_text'], axis=1)
counts = []
categories = list(df_toxic.columns.values)

NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, y_train[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test[category], prediction)))

... Processing toxic
Test accuracy is 0.9405238828967643
... Processing severe_toxic
Test accuracy is 0.9035439137134053
... Processing obscene
Test accuracy is 0.737442218798151
... Processing threat
Test accuracy is 0.9694915254237289
... Processing insult
Test accuracy is 0.6807395993836671
... Processing identity_hate
Test accuracy is 0.9124807395993837


In [41]:
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, y_train[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test[category], prediction)))

... Processing toxic
Test accuracy is 0.9380585516178737
... Processing severe_toxic
Test accuracy is 0.8970724191063174
... Processing obscene
Test accuracy is 0.7895223420647149
... Processing threat
Test accuracy is 0.9738058551617874
... Processing insult
Test accuracy is 0.6844375963020031
... Processing identity_hate
Test accuracy is 0.92326656394453


In [40]:
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, y_train[category])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test[category], prediction)))

... Processing toxic
Test accuracy is 0.9405238828967643
... Processing severe_toxic
Test accuracy is 0.903235747303544
... Processing obscene
Test accuracy is 0.8024653312788906
... Processing threat
Test accuracy is 0.9725731895223421
... Processing insult
Test accuracy is 0.711864406779661
... Processing identity_hate
Test accuracy is 0.9198767334360555
